In [1133]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
import joblib                                                                                                                                            
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import auc
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import KFold
from scipy.stats import pearsonr



In [1134]:

fajiao = pd.read_table("../data-2/fajiao-condition-4.11.txt",sep='\t',index_col=0)
fajiao = pd.get_dummies(fajiao, columns=fajiao.columns) 

fajiao['label'] = fajiao.groupby(fajiao.columns.tolist()).ngroup()


fajiao_drop = fajiao.drop_duplicates()
fajiao_drop = fajiao_drop.drop('label', axis=1)

X = np.array(fajiao_drop)
# X

# X  = StandardScaler().fit_transform(X)
X  = Normalizer().fit_transform(X)
# # X  = MinMaxScaler().fit_transform(X)
# X  = StandardScaler().fit_transform(X)
X.shape

(13, 17)

In [1135]:
data = pd.read_table('../data-2/total4.15.txt',sep='\t',index_col=0)
data_fjg= data.loc[:, ~data.columns.str.contains('BC|NT|MC|PC')].T
data_fjg.columns = data_fjg.iloc[0]
data_fjg = data_fjg[1:]
data_fjg = data_fjg.sort_index()
data_fjg = data_fjg.replace("#DIV/0!", 0)
data_fjg = data_fjg.apply(pd.to_numeric)
data_fjg.fillna(0, inplace=True) 
# data_fjg


data_fjg.index = data_fjg.index.astype(float) 
data_fjg = data_fjg.sort_index() 
selected_columns = ['1000315-10-3', '1000150-45-1', '1000315-10-4', '1000149-94-5', '622-78-6', '1000152-34-3', '1000162-57-7']
data_fjg = data_fjg.loc[:, selected_columns]
# data_fjg

data_fjg['label'] = fajiao['label']
fajiao_mean = data_fjg.groupby('label').mean()

fajiao_mean_unique = fajiao_mean.reset_index().drop_duplicates()
fajiao_mean_unique = fajiao_mean_unique.drop('label', axis=1)
fajiao_mean_unique

Y = np.array(fajiao_mean_unique)
# 
Y  = Normalizer().fit_transform(Y)
# Y  = MinMaxScaler().fit_transform(Y)
# Y  = StandardScaler().fit_transform(Y)
Y.shape

(13, 7)

In [1136]:

base_model = GradientBoostingRegressor(n_estimators=10, random_state=42,subsample=0.5,learning_rate=0.01) #
gbr_model = MultiOutputRegressor(base_model)

In [1137]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

MSE = []
RMSE = []
MAE = []
PCC = []
R2 = []
adjusted_r2 = []

MSE_train = []
RMSE_train = []
MAE_train = []
PCC_train = []
R2_train = []
adjusted_r2_train = []


for train_idx, test_idx in cv.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, y_test = Y[train_idx], Y[test_idx]
    gbr_model.fit(X_train, Y_train)
    # print(X_train.shape, Y_train.shape)
    #train
    x_pred = gbr_model.predict(X_train)
    mse = mean_squared_error(Y_train, x_pred)
    rmse = np.sqrt(mean_squared_error(Y_train, x_pred))
    mae = np.mean(np.abs(Y_train - x_pred))
    correlation_array = np.empty((len(Y_train), 1))
    for i in range(len(Y_train)):
        correlation = np.corrcoef(Y_train[i], x_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(Y_train, x_pred)
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(Y_train) - 1) / (len(Y_train) - num_features - 1)
    MSE_train .append(mse)
    RMSE_train .append(rmse)
    MAE_train .append(mae)
    PCC_train .append(pcc)
    R2_train .append(r2)
    adjusted_r2_train .append(adj_r2)

    #test
    y_pred = gbr_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = np.mean(np.abs(y_test - y_pred))
    correlation_array = np.empty((len(y_test), 1))
    for i in range(len(y_test)):
        correlation = np.corrcoef(y_test[i], y_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(y_test, y_pred) 
    num_features = X.shape[1]
    adj_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)
    MSE.append(mse)
    RMSE.append(rmse)
    MAE.append(mae)
    PCC.append(pcc)
    R2.append(r2)
    adjusted_r2.append(adj_r2)


for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE_train, RMSE_train,MAE_train, PCC_train,R2_train,adjusted_r2_train), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE_train) / len(MSE_train)
average_RMSE = sum(RMSE_train) / len(RMSE_train)
average_MAE= sum(MAE_train) / len(MAE_train)
average_PCC = sum(PCC_train) / len(PCC_train)
average_R2 = sum(R2_train) / len(R2_train)
average_adjusted_r2 = sum(adjusted_r2_train) / len(adjusted_r2_train)

print(f'Average MSE_train = {average_MSE:.7f}')
print(f'Average RMSE_train = {average_RMSE:.7f}')
print(f'Average MAE_train = {average_MAE:.7f}')
print(f'Average PCC_train = {average_PCC:.7f}')
# print(f'Average R2_train = {average_R2:.7f}')
# print(f'Average adjusted_R2_train = {average_adjusted_r2:.7f}')



for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE, RMSE,MAE, PCC,R2,adjusted_r2), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE) / len(MSE)
average_RMSE = sum(RMSE) / len(RMSE)
average_MAE= sum(MAE) / len(MAE)
average_PCC = sum(PCC) / len(PCC)
average_R2 = sum(R2) / len(R2)
average_adjusted_r2 = sum(adjusted_r2) / len(adjusted_r2)

print(f'Average MSE = {average_MSE:.7f}')
print(f'Average RMSE = {average_RMSE:.7f}')
print(f'Average MAE = {average_MAE:.7f}')
print(f'Average PCC = {average_PCC:.7f}')
# print(f'Average R2 = {average_R2:.7f}')
# print(f'Average adjusted_R2 = {average_adjusted_r2:.7f}')

Fold 1: MSE = 0.0390, RMSE = 0.1974,MAE = 0.1225, PCC = 0.7966, R2 = 0.0870, adjusted_r2 = -1.7390
Fold 2: MSE = 0.0371, RMSE = 0.1926,MAE = 0.1212, PCC = 0.8112, R2 = 0.0893, adjusted_r2 = -1.7320
Fold 3: MSE = 0.0357, RMSE = 0.1890,MAE = 0.1196, PCC = 0.8091, R2 = 0.0797, adjusted_r2 = -1.7608
Fold 4: MSE = 0.0372, RMSE = 0.1928,MAE = 0.1150, PCC = 0.8036, R2 = 0.0741, adjusted_r2 = -1.3147
Fold 5: MSE = 0.0410, RMSE = 0.2025,MAE = 0.1289, PCC = 0.7828, R2 = 0.0782, adjusted_r2 = -1.3046
Average MSE_train = 0.0379948
Average RMSE_train = 0.1948672
Average MAE_train = 0.1214206
Average PCC_train = 0.8006665
Fold 1: MSE = 0.0370, RMSE = 0.1924,MAE = 0.1281, PCC = 0.8065, R2 = -0.0304, adjusted_r2 = 1.1374
Fold 2: MSE = 0.0468, RMSE = 0.2163,MAE = 0.1193, PCC = 0.7324, R2 = -0.1533, adjusted_r2 = 1.1538
Fold 3: MSE = 0.0566, RMSE = 0.2378,MAE = 0.1553, PCC = 0.7181, R2 = -88.0179, adjusted_r2 = 12.8690
Fold 4: MSE = 0.0601, RMSE = 0.2452,MAE = 0.1550, PCC = 0.6537, R2 = -5.6422, adjuste

In [945]:

for i in np.arange(0.1,0.9,0.05):
# for i in np.arange(2,25,1):
    print(i)
    base_model = GradientBoostingRegressor(n_estimators=10, random_state=42,learning_rate=0.01) #,subsample=i,learning_rate=0.35
    gbr_model = MultiOutputRegressor(base_model)
    gbr_model.fit(X_train, Y_train)
    y_pred = gbr_model.predict(X_test)


    # rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    # print("RMSE:", rmse)
    # r2 = r2_score(y_test, y_pred) 
    # print("R^2:", r2)
    correlation_array = np.empty((len(y_test), 1))
    for i in range(len(y_test)):
        correlation = np.corrcoef(y_test[i], y_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    print("PPC:", pcc)


0.1
PPC: 0.8216931482761227
0.15000000000000002
PPC: 0.8216931482761227
0.20000000000000004
PPC: 0.8216931482761227
0.25000000000000006
PPC: 0.8216931482761227
0.30000000000000004
PPC: 0.8216931482761227
0.3500000000000001
PPC: 0.8216931482761227
0.40000000000000013
PPC: 0.8216931482761227
0.45000000000000007
PPC: 0.8216931482761227
0.5000000000000001
PPC: 0.8216931482761227
0.5500000000000002
PPC: 0.8216931482761227
0.6000000000000002
PPC: 0.8216931482761227
0.6500000000000001
PPC: 0.8216931482761227
0.7000000000000002
PPC: 0.8216931482761227
0.7500000000000002
PPC: 0.8216931482761227
0.8000000000000002
PPC: 0.8216931482761227
0.8500000000000002
PPC: 0.8216931482761227


In [1129]:

yp = gbr_model.predict(X)
# np.savetxt('my_array.txt', yp, delimiter='\t')
yp.shape  

(13, 7)

In [1132]:
yp
result = pd.DataFrame(yp,index = fajiao_drop.index,columns=selected_columns)
result

,1000315-10-3,1000150-45-1,1000315-10-4,1000149-94-5,622-78-6,1000152-34-3,1000162-57-7
4.2,0.211120,0.238194,0.000000,0.641986,0.931331,0.915341,0.657212
5.2,0.421379,0.486664,0.262238,0.143350,0.670052,0.177917,0.285347
6.2,0.212001,1.000000,0.788782,0.055225,0.249470,0.153550,0.506070
16.1,0.194334,0.232175,0.058689,0.812833,0.883172,0.991805,0.995803
17.1,0.162520,0.000000,0.043033,1.000000,0.904170,1.000000,1.000000
23.1,0.993661,0.280340,0.788219,0.006327,0.013700,0.100912,0.134122
24.1,1.000000,0.532921,0.799265,0.011115,0.000000,0.126415,0.269918
25.1,0.373885,0.002967,0.307857,0.300495,0.641458,0.243116,0.329602
26.1,0.000000,0.262514,0.005462,0.740137,1.000000,0.260329,0.179463
27.1,0.367267,0.013631,0.842753,0.489231,0.157926,0.000000,0.000000


In [1131]:

yp  = Normalizer().fit_transform(yp)
yp = MinMaxScaler().fit_transform(yp)
print(yp.shape)
loaded_model = joblib.load("../code-2/result/GBDT_Classifier-7f.joblib")
y_pred = loaded_model.predict(yp).tolist() 
y_pred

(13, 7)


[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]

In [1109]:

df13 = fajiao.drop_duplicates()

df54 = pd.read_csv("fjg_label.csv",index_col=0)
df54['code'] = fajiao['label']
df54['mean'] = df54.groupby('code')['label'].transform('mean')
df54['mean_label'] = np.where(df54['mean'] > 0.6, 1, 0)
df54

df54 = df54[df54.index.isin(df13.index)]
truth_simi = df54['mean_label'].tolist()
truth_simi

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [1120]:
print('precision（test）:',metrics.precision_score(truth_simi,y_pred))
print('recall（test）:',metrics.recall_score(truth_simi,y_pred))
print('Accuracy（test）:',metrics.accuracy_score(truth_simi,y_pred))
# print('F1 （test）:',f1_score(truth_simi,y_pred))
print("y_test:",truth_simi,"\ny_pred:",y_pred)
fpr, tpr, thresholds = roc_curve(truth_simi, y_pred)
auc_1 = auc(fpr, tpr)
print("AUC : ", auc_1)

precision（测试集）: 0.25
recall（测试集）: 1.0
Accuracy（测试集）: 0.7692307692307693
y_test: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 
y_pred: [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
AUC :  0.875
